In [1]:
import numpy as np

## Further Study

### Closed Form

Gradient descent gives one possible mean for minimizing $J$, which uses iterative approach and may take time.  In the situation where we know that our cost function is strictly concave or convex, we can explicitly take its derivative to zero.  This process of such derivation is called obtaining the **normal equations** or **closed form**. 

The **closed form** of linear regression can be derived easily.  Let $\mathbf{X}$ be a matrix of shape $(m, n)$, $\boldsymbol{\theta}$ as shape $(n, )$, and $\mathbf{y}$ as vector of shape $(m, )$.  Instead of writing the cost function as power of square, we shall write it in matrix multiplication as follows:

$$\frac{\partial J}{\partial \boldsymbol{\theta}} (\mathbf{X}
 - \mathbf{y})^T*(\mathbf{X}
-\mathbf{y})$$

Recall the following properties:

$$\frac{\partial J}{\partial \mathbf{X}} \mathbf{X}^T\mathbf{X}=2\mathbf{X} \tag{A}$$
$$\frac{\partial J}{\partial \mathbf{X}} \mathbf{A}\mathbf{X}=\mathbf{A}^T$$
$$(\mathbf{X}\mathbf{y})^T = \mathbf{y}^T\mathbf{X}^T$$

Therefore

\begin{align*}
\frac{\partial J}{\partial \boldsymbol{\theta}} (\mathbf{X}\boldsymbol{\theta} - \mathbf{y})^T*(\mathbf{X}\boldsymbol{\theta}-\mathbf{y}) &= \frac{\partial J}{\partial \boldsymbol{\theta}} (\boldsymbol{\theta}^T\mathbf{X}^T\mathbf{X}\boldsymbol{\theta} - \boldsymbol{\theta}^T\mathbf{X}^T\mathbf{y} - \mathbf{y}^T\mathbf{X}\boldsymbol{\theta} + \mathbf{y}^T\mathbf{y})\\
&= 2\mathbf{X}^T\mathbf{X}\boldsymbol{\theta} - 2\mathbf{X}^T\mathbf{y} \tag{see note*}\\
&= \mathbf{X}^T\mathbf{X}\boldsymbol{\theta} - \mathbf{X}^T\mathbf{y} = 0 \tag{set derivative to 0}\\
&= \boldsymbol{\theta} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}
\end{align*}


Note*: Since $\mathbf{X}\boldsymbol{\theta}$ is a vector, and so is $\mathbf{y}$, it doesn't matter what the order is, thus we can simply add them to 2.  Also, we got 2 in front of the first part because we have two $\theta$ (used the property A)


**Why not closed form always**.  The answer is simple.  It does not always exists or possible, for example, the cost function is not convex or concave.  But of course, if it exists, we usually prefer closed form given that it is usually faster than gradient descent.  Nevertheless, as you can see, taking inverse of huge number of features can be expensive, thus it is also not always straightforward thing to always prefer closed form.

Yes, that's it for most of the theoretical stuff.  Let's start implementing some of these concepts so we can better understand them.

The closed form is a normal equations derived from setting the derivatives = 0.  By performing only some inverse operations and matrix multiplication, we will be able to get the theta.

$$\boldsymbol{\theta} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$$

When closed form is available, is doable (can be inversed - can use pseudoinverse), and with not many features (i.e., inverse can be slow), it is recommended to always use closed form.  

**Implementation steps:**

1. Prepare your data
    - add intercept
    - $\mathbf{X}$ and $\mathbf{y}$ and $\mathbf{w}$ in the right shape
        - $\mathbf{X}$ -> $(m, n)$
        - $\mathbf{y}$ -> $(m, )$
        - $\mathbf{w}$ -> $(n, )$
        - where $m$ is number of samples
        - where $n$ is number of features
    - train-test split
    - feature scale
    - clean out any missing data
    - (optional) feature engineering
2. Plug everything into the equation.  Here we shall use X_train to retrieve the $\boldsymbol{\theta}$
$$\boldsymbol{\theta} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$$

3. We simply using the $\boldsymbol{\theta}$, we can perform a dot product with our X_test which will give us $\mathbf{\hat{y}}$.

4. We then calculate the errors using mean-squared-error function:

$$\frac{1}{m}\sum_{i=1}^m(h_\theta(x^{(i)}) - y^{(i)})^2$$

Note that it's a bit different from our $J(\boldsymbol{\theta})$ because $J(\boldsymbol{\theta})$ puts $\frac{1}{2}$ instead of $\frac{1}{m}$ for mathematical convenience for derivatives, since we know changing constants do not change the optimization results.


Let's implement.

### Step 1. Prepare data

In [2]:
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from time import time

diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target
m = X.shape[0]  #number of samples
n = X.shape[1]  #number of features

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

# actually you can do like this too
# X = np.insert(X, 0, 1, axis=1)
intercept = np.ones((X_train.shape[0], 1))
X_train   = np.concatenate((intercept, X_train), axis=1)
intercept = np.ones((X_test.shape[0], 1))
X_test    = np.concatenate((intercept, X_test), axis=1)

### Step 2: Fit your algorithm 

#### 1. Define your algorithm

In [3]:
from numpy.linalg import inv

# order of operation DOES NOT MATTER
# But don't flip y before X^T for example
def closed_form(X, y):
    return inv(X.T @ X) @ X.T @ y

In [4]:
# let's use the closed_form to find the theta
theta = closed_form(X_train, y_train)
theta  #<------this is our model

array([153.23624595,  -0.51349008, -10.42423616,  25.40910714,
        14.53692679, -38.95536033,  26.35053758,   5.38749134,
         6.1627566 ,  36.72524689,   1.84754747])

#### 2. Compute accuracy/loss

In [5]:
# Compute the accuracy/loss

yhat = X_test @ theta #==> X (m, n+1)  @ (n+1, ) w ==> (m, ) y

# if I want to compare yhat and y, I need to make sure they are the same shape
assert y_test.shape == yhat.shape

In [6]:
# get the mse
mse = ((y_test - yhat)**2).sum() / X_test.shape[0]
print("Mean squared errors: ", mse)

Mean squared errors:  2324.9072894171472
